# Bills and votes

This notebook aims to
1. generate csv

In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
import requests


In [2]:
ALL_LAWS_PATH = '../data/all-votes/laws_20years.csv'
VOTES_115 = '../data/all-votes/votes_115.csv'
API_KEY = 'EaNt0652GV92i9U9Mlhs0ggCwLPyRB23bc6qAeyX'


In [3]:
all_laws = pd.read_csv(ALL_LAWS_PATH)

In [4]:
def get_cosponsors_idx(bills):
    bills['cosponsors_sen'] = ''
    bills['cosponsors_rep'] = ''
    bills['cosponsors_del'] = ''

    for i in range(len(bills)):
        cosponsors_list_sen = []
        cosponsors_list_rep = []
        cosponsors_list_del = []
        for j in range(len(bills.cosponsors.iloc[i])):
            if bills.cosponsors.iloc[i][j]['cosponsor_title'] == 'Sen.':
                cosponsors_list_sen.append(bills.cosponsors.iloc[i][j]['cosponsor_id'])
            elif bills.cosponsors.iloc[i][j]['cosponsor_title'] == 'Rep.':
                cosponsors_list_rep.append(bills.cosponsors.iloc[i][j]['cosponsor_id'])
            elif bills.cosponsors.iloc[i][j]['cosponsor_title'] == 'Del.':
                cosponsors_list_del.append(bills.cosponsors.iloc[i][j]['cosponsor_id'])
        bills.cosponsors_sen.iloc[i] = cosponsors_list_sen
        bills.cosponsors_rep.iloc[i] = cosponsors_list_rep
        bills.cosponsors_del.iloc[i] = cosponsors_list_del
        
    bills = bills.drop('cosponsors', axis = 1)

    return bills

In [22]:
def gen_bill_csv_file(congress_number):
    bills = all_laws[(all_laws['congress'] == congress_number) & (all_laws['bill_api_uri'].notnull())]
    bills_ = bills['bill_api_uri'].unique()

    bills_sponsorship = []

    for i in range(0, len(bills_)):
        url = bills_[i][:-5] + '/cosponsors.json'

        d = {}
        try:
            req = requests.get(url, headers={'X-API-Key': API_KEY}).json()
            results = req['results'][0]
            d['bill_url'] = bills_[i]
            d['sponsor_party'] = results['sponsor_party']
            d['sponsor_id'] = results['sponsor_id']
            d['sponsor_title'] = results['sponsor_title']
            d['committees'] = results['committees']
            d['number_of_cosponsors'] = results['number_of_cosponsors']
            d['cosponsors_by_party'] = results['cosponsors_by_party']
            d['cosponsors'] = results['cosponsors']

            bills_sponsorship.append(d)
        except:
            pass

    bills_sponsorship = pd.DataFrame(bills_sponsorship)
    bills = bills.merge(bills_sponsorship, left_on = 'bill_api_uri', right_on = 'bill_url')
    bills = bills.drop(['result','total_no', 'total_not_voting', 'total_yes', 'cosponsors_by_party','number_of_cosponsors','congress'], axis=1)
    bills = bills[bills['sponsor_id'] !='']
    bills = bills[bills['sponsor_title'] !='']
    bills = bills[bills['sponsor_party'] != 'ID']
    bills = get_cosponsors_idx(bills)
    bills.to_csv('../data/bills/bills_{}.csv'.format(congress_number), index = False)



In [23]:
for congress_number in range(105, 116):
    gen_bill_csv_file(congress_number);

## Get votes senate

In [26]:
def get_votes(congress_number):
    votes = list(all_laws[all_laws.congress == congress_number]['vote_uri'])

    df = pd.DataFrame()

    for i, url in enumerate(votes):
        req = requests.get(url, headers={'X-API-Key': API_KEY}).json()
        d = pd.DataFrame(req['results']['votes']['vote']['positions'])
        d['vote_uri'] = url

        df = pd.concat([df, d])

    df.to_csv('../data/votes/votes_{}.csv'.format(congress_number), index=False)

In [37]:
for congress_number in range(105, 116):
    get_votes(congress_number);